In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import math
from itertools import combinations
from numpy import linalg as LA
import random

In [2]:
# Create a model
m = Model("VC")

Restricted license - for non-production use only - expires 2023-10-25


In [3]:
# Aircrafts Setting
# distance unit :(km)

# ( Radius Setting r = sqrt(123^2+123^2) )

r = math.sqrt(123**2+123**2)

# Situation 1
#Aircrafts = [[-123,-123,45],[123,-123,135]]

# Situation 2
#Aircrafts = [[-123,-123,30],[123,-123,150]]

# Situation 3
Aircrafts = [[-123,-123,60],[123,-123,135]]


# Situation 4(random)  
'''
k = random.randint(0,90)
a = random.randint(-30,30)
g = random.randint(0,90)
b = random.randint(-30,30)
l = random.randint(0,90)
c = random.randint(-30,30)
Aircrafts = [[r*math.sin(k),r*math.cos(k),k+a],[r*math.sin(g),r*math.cos(g),g+b],[r*math.sin(l),r*math.cos(l),l+c]]'''
'''
Ki=[]
for i in range(random.randint(2,5)):
    k = random.randint(0,90)
    X=r*math.sin(k)
    Y=r*math.cos(k)
    if X>=0 and Y>=0:
        φ = random.randint(-180,-90)
    elif X<=0 and Y>=0:
        φ = random.randint(-90,0)
    elif X<=0 and Y<=0:
        φ = random.randint(0,90)
    else:
        φ = random.randint(90,180)    
    Ki.append([X,Y,φ])
    
Aircrafts = []
for j in range(len(Ki)):
    Aircrafts.append(Ki[j])'''
    



# number of aircrafts
n = len(Aircrafts)

# create pairing list
num_aircraft_list = []
for i in  range(n):
    num_aircraft_list.append(i)

Aircraft_index_comb = list(combinations(num_aircraft_list,2))

# number of combinations
b = len(Aircraft_index_comb)

print(num_aircraft_list, ", i index set")
print(Aircraft_index_comb, ", (i,j) index set")

# safety distance
d = 8

[0, 1] , i index set
[(0, 1)] , (i,j) index set


In [4]:
Aircrafts

[[-123, -123, 60], [123, -123, 135]]

# Variables

In [5]:
Wi = m.addVars(b,vtype=GRB.BINARY)
Yi =  m.addVars(b,vtype=GRB.BINARY)
Zi = m.addVars(b,vtype=GRB.BINARY)
Vi = m.addVars(n,vtype=GRB.CONTINUOUS)

In [6]:
Xi = []
for i in range(b):
    Xi.append([ Aircrafts[ Aircraft_index_comb[i][0] ][0] - Aircrafts[ Aircraft_index_comb[i][1] ][0] 
               , Aircrafts[ Aircraft_index_comb[i][0] ][1] - Aircrafts[ Aircraft_index_comb[i][1] ][1] ])

In [7]:
Vri = []
for i in range(b):
    Vri.append([ math.cos(math.radians(Aircrafts[ Aircraft_index_comb[i][0] ][2]))*Vi[Aircraft_index_comb[i][0]] - math.cos(math.radians(Aircrafts[ Aircraft_index_comb[i][1] ][2]))*Vi[Aircraft_index_comb[i][1]]  
               ,  math.sin(math.radians(Aircrafts[ Aircraft_index_comb[i][0] ][2]))*Vi[Aircraft_index_comb[i][0]] - math.sin(math.radians(Aircrafts[ Aircraft_index_comb[i][1] ][2]))*Vi[Aircraft_index_comb[i][1]]])

# Objective Function

In [8]:
m.setObjective( quicksum(Wi[i] for i in range(b)) ,GRB.MAXIMIZE)

# Constraints

In [9]:
max_speed = 400*1.852*1.03
min_speed = 400*1.852*0.94
m.addConstrs( Vi[i] >= min_speed for i in range(n) ) 
m.addConstrs( Vi[i] <= max_speed for i in range(n) ) 
m.update()

In [10]:
m.addConstrs(
    (Xi[i][0] * Vri[i][0] + Xi[i][1] * Vri[i][1]) *
    (Xi[i][0] * Vri[i][0] + Xi[i][1] * Vri[i][1]) -
    ((Vri[i][0] * Vri[i][0] + Vri[i][1] * Vri[i][1])) *
    ((Xi[i][0] * Xi[i][0] + Xi[i][1] * Xi[i][1]) - d * d) <= 1000000 *
    Zi[i] for i in range(b))
m.update()

In [11]:
m.addConstrs(
    (Xi[i][0] * Vri[i][0] + Xi[i][1] * Vri[i][1]) *
    (Xi[i][0] * Vri[i][0] + Xi[i][1] * Vri[i][1]) -
    ((Vri[i][0] * Vri[i][0] + Vri[i][1] * Vri[i][1])) *
    ((Xi[i][0] * Xi[i][0] + Xi[i][1] * Xi[i][1]) - d * d) >= 1000000*
    (Zi[i] - 1) for i in range(b))
m.update()

In [12]:
m.addConstrs(
    (Xi[i][0] * Vri[i][0] + Xi[i][1] * Vri[i][1]) <= 1000000 * Yi[i]
    for i in range(b))
m.update()

In [13]:
m.addConstrs(
    (Xi[i][0] * Vri[i][0] + Xi[i][1] * Vri[i][1]) >= 1000000 * (Yi[i] - 1)
    for i in range(b))
m.update()

In [14]:
m.addConstrs(Wi[i] >= Yi[i] for i in range(b))
m.update()

In [15]:
m.addConstrs(Wi[i] >= Zi[i] for i in range(b))
m.update()

In [16]:
m.addConstrs((Yi[i] + Zi[i]) >= Wi[i] for i in range(b))
m.update()

In [17]:
m.Params.NonConvex = 2
m.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 9 rows, 5 columns and 17 nonzeros
Model fingerprint: 0xb052ca5f
Model has 2 quadratic constraints
Variable types: 2 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [3e+04, 7e+04]
  QLMatrix range   [1e+06, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+02, 1e+06]
  QRHS range       [1e+06, 1e+06]
Presolve removed 9 rows and 2 columns
Presolve time: 0.00s
Presolved: 12 rows, 6 columns, 32 nonzeros
Presolved model has 3 bilinear constraint(s)
Variable types: 5 continuous, 1 integer (1 binary)

Root relaxation: infeasible, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestB

In [18]:
for i in range(b):
    print(Yi[i])  

<gurobi.Var C1>


In [19]:
for i in range(b):
    print(Zi[i])     

<gurobi.Var C2>


In [20]:
for i in range(n):
    print(Vi[i])


<gurobi.Var C3>
<gurobi.Var C4>


In [21]:
m.computeIIS()
m.write("model1.ilp")


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0        11         -         0         2         -           0s
        3         3         -         1         1         -           0s

IIS computed: 3 constraints, 1 bounds
IIS runtime: 0.04 seconds (0.00 work units)


## HAC

In [22]:
HAC = Model("HAC")

## Variables

In [23]:
theta = HAC.addVars(n, vtype=GRB.CONTINUOUS)
yij = HAC.addVars(b, vtype=GRB.BINARY)
tij = HAC.addVars(b, vtype=GRB.CONTINUOUS)
AngleSum = HAC.addVars(n, vtype=GRB.CONTINUOUS)
Cos_AngleSum = HAC.addVars(n, vtype=GRB.CONTINUOUS)
Sin_AngleSum = HAC.addVars(n, vtype=GRB.CONTINUOUS)
Vri_2 = HAC.addVars(b, vtype=GRB.CONTINUOUS)

In [24]:
HAC.setObjective( quicksum(theta[i]**2 for i in range(n)) ,GRB.MINIMIZE )

## Constructing elements of Constraints( pre constraint )

In [25]:
HAC.addConstrs(AngleSum[i] == (Aircrafts[i][2] +
                               theta[i]) for i in range(n))
HAC.update()

In [26]:
HAC.addGenConstrCos(AngleSum[i], Cos_AngleSum[i])
HAC.update()

In [27]:
HAC.addGenConstrSin(AngleSum[i], Sin_AngleSum[i])
HAC.update()

In [28]:
Xi = []
for i in range(b):
    Xi.append([ Aircrafts[ Aircraft_index_comb[i][0] ][0] - Aircrafts[ Aircraft_index_comb[i][1] ][0] 
               , Aircrafts[ Aircraft_index_comb[i][0] ][1] - Aircrafts[ Aircraft_index_comb[i][1] ][1] ])

In [29]:
try:
    Vri = []
    for i in range(b):
        Vri.append([
            Cos_AngleSum[Aircraft_index_comb[i][0]] *
            Vi[Aircraft_index_comb[i][0]].x -
            Cos_AngleSum[Aircraft_index_comb[i][1]] *
            Vi[Aircraft_index_comb[i][1]].x,
            Sin_AngleSum[Aircraft_index_comb[i][0]] *
            Vi[Aircraft_index_comb[i][0]].x -
            Sin_AngleSum[Aircraft_index_comb[i][1]] *
            Vi[Aircraft_index_comb[i][1]].x
        ])
except Exception:
    Vi=[]
    for i in range(n):
        Vi.append(400*1.852)

    #### Try to change the speed!!!
    Vri = []
    for i in range(b):
        Vri.append([
            Cos_AngleSum[Aircraft_index_comb[i][0]] *
            Vi[Aircraft_index_comb[i][0]] -
            Cos_AngleSum[Aircraft_index_comb[i][1]] *
            Vi[Aircraft_index_comb[i][1]],
            Sin_AngleSum[Aircraft_index_comb[i][0]] *
            Vi[Aircraft_index_comb[i][0]] -
            Sin_AngleSum[Aircraft_index_comb[i][1]] *
            Vi[Aircraft_index_comb[i][1]]
        ])


In [30]:
'''
Vi=[]
for i in range(n):
    Vi.append(400*1.852)


Vri = []
for i in range(b):
    Vri.append([
        Cos_AngleSum[Aircraft_index_comb[i][0]] *
        Vi[Aircraft_index_comb[i][0]] -
        Cos_AngleSum[Aircraft_index_comb[i][1]] *
        Vi[Aircraft_index_comb[i][1]],
        Sin_AngleSum[Aircraft_index_comb[i][0]] *
        Vi[Aircraft_index_comb[i][0]] -
        Sin_AngleSum[Aircraft_index_comb[i][1]] *
        Vi[Aircraft_index_comb[i][1]]
    ])'''

'\nVi=[]\nfor i in range(n):\n    Vi.append(400*1.852)\n\n\nVri = []\nfor i in range(b):\n    Vri.append([\n        Cos_AngleSum[Aircraft_index_comb[i][0]] *\n        Vi[Aircraft_index_comb[i][0]] -\n        Cos_AngleSum[Aircraft_index_comb[i][1]] *\n        Vi[Aircraft_index_comb[i][1]],\n        Sin_AngleSum[Aircraft_index_comb[i][0]] *\n        Vi[Aircraft_index_comb[i][0]] -\n        Sin_AngleSum[Aircraft_index_comb[i][1]] *\n        Vi[Aircraft_index_comb[i][1]]\n    ])'

## Constraints

In [31]:
theta_Max = 30
theta_Min = -30
HAC.addConstrs(theta[i] <= theta_Max for i in range(n))
HAC.addConstrs(theta[i] >= theta_Min for i in range(n))
HAC.update()

In [32]:
HAC.addConstrs( (Vri[i][0]*Vri[i][0]+Vri[i][1]*Vri[i][1])*( (Xi[i][0]*Xi[i][0]+Xi[i][1]*Xi[i][1])  - d*d ) - (Xi[i][0]*Vri[i][0]+Xi[i][1]*Vri[i][1])*(Xi[i][0]*Vri[i][0]+Xi[i][1]*Vri[i][1]) <= 10000*yij[i] for i in range(b) )
HAC.update()

In [33]:
HAC.addConstrs( (Vri[i][0]*Vri[i][0]+Vri[i][1]*Vri[i][1])*( (Xi[i][0]*Xi[i][0]+Xi[i][1]*Xi[i][1])  - d*d ) - (Xi[i][0]*Vri[i][0]+Xi[i][1]*Vri[i][1])*(Xi[i][0]*Vri[i][0]+Xi[i][1]*Vri[i][1]) >= 10000*(yij[i]-1) for i in range(b) )
HAC.update()

In [34]:
HAC.addConstrs(Vri_2[i] == Vri[i][0]*Vri[i][0]+Vri[i][1]*Vri[i][1] for i in range(b))
HAC.update()

In [35]:
HAC.addConstrs(-1*(Xi[i][0]*Vri[i][0]+Xi[i][1]*Vri[i][1]) <= 10000*yij[i]*Vri_2[i]  for i in range(b))
HAC.update()

In [36]:
HAC.addConstrs(-1*(Xi[i][0]*Vri[i][0]+Xi[i][1]*Vri[i][1]) >= 10000*(yij[i]-1)*Vri_2[i] for i in range(b))
HAC.update()

In [37]:
HAC.params.NonConvex = 2

Set parameter NonConvex to value 2


In [38]:
HAC.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 6 rows, 11 columns and 8 nonzeros
Model fingerprint: 0x2923fff2
Model has 2 quadratic objective terms
Model has 5 quadratic constraints
Model has 2 general constraints
Variable types: 10 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+04, 7e+10]
  QLMatrix range   [1e+00, 2e+05]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 1e+02]
  QRHS range       [1e+04, 1e+04]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 38 rows and 625 columns
Presolve time: 0.02s
Presolved: 71 rows, 646 columns, 10811 nonzeros
Presolved model has 2 SOS constraint(s)
Presolved model has 1 quadratic objective terms
Presolved model has 6 bilin

In [39]:
for i in range(n):
    print(theta[i])

<gurobi.Var C0 (value 0.0)>
<gurobi.Var C1 (value 3.230076757950883)>
